In [97]:
# Import pandas
import pandas as pd

# Assign spreadsheet filename: file
file = 'imd_student_blind.xlsx'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Print sheet names
print(xl.sheet_names)

['Sheet1']


In [98]:
# Load a sheet into a DataFrame by index: df
df = xl.parse(0)

# Print the head of the DataFrame df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


In [99]:
df.columns

Index(['a_ID', 'CEP', 'ano_ingresso', 'periodo_ingresso', 'status',
       'ano_disciplina', 'periodo_disciplina', 'nota', 'disciplina_ID',
       'status.disciplina'],
      dtype='object')

In [86]:
df.shape

(4842, 10)

In [6]:
disciplina_0 = df[df['disciplina_ID'] == 0]

In [182]:
#Cópia por valores, não mexer no DF principal
import copy
disciplinas = copy.copy(df)
disciplinas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4842 entries, 0 to 4841
Data columns (total 10 columns):
a_ID                  4842 non-null int64
CEP                   4842 non-null int64
ano_ingresso          4842 non-null int64
periodo_ingresso      4842 non-null int64
status                4842 non-null object
ano_disciplina        4842 non-null int64
periodo_disciplina    4842 non-null int64
nota                  4842 non-null float64
disciplina_ID         4842 non-null int64
status.disciplina     4842 non-null object
dtypes: float64(1), int64(7), object(2)
memory usage: 416.1+ KB


In [183]:
disciplinas.columns.values.tolist()

['a_ID',
 'CEP',
 'ano_ingresso',
 'periodo_ingresso',
 'status',
 'ano_disciplina',
 'periodo_disciplina',
 'nota',
 'disciplina_ID',
 'status.disciplina']

In [184]:
#Retira informações desnecessárias para essa análise

disciplinas.drop(disciplinas.columns[0:4], axis=1, inplace=True)

In [185]:
#Objetivo 1 -> Média da disciplina X por período e ano
#Objetivo 2 -> Taxa de trancamento e cancelamento por período e ano
disciplinas.columns.values.tolist()

['status',
 'ano_disciplina',
 'periodo_disciplina',
 'nota',
 'disciplina_ID',
 'status.disciplina']

In [189]:
disciplinas = disciplinas[disciplinas['status'] == 'ATIVO']
disciplinas.head()

,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
3,ATIVO,2014,2,6.1,0,Aprovado
4,ATIVO,2014,1,3.0,0,Reprovado
5,ATIVO,2014,2,7.2,0,Aprovado
8,ATIVO,2014,2,8.9,0,Aprovado
12,ATIVO,2014,2,5.6,0,Aprovado


In [207]:
disciplinas_categorias =  disciplinas.groupby(['ano_disciplina', 'periodo_disciplina', 'disciplina_ID']).mean()

# convert the index to a column
disciplinas_categorias.reset_index(inplace = True )
disciplinas_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
ano_disciplina        42 non-null int64
periodo_disciplina    42 non-null int64
disciplina_ID         42 non-null int64
nota                  42 non-null float64
dtypes: float64(1), int64(3)
memory usage: 1.4 KB


In [219]:
disciplinas_categorias[disciplinas_categorias['disciplina_ID'] == 0]

,ano_disciplina,periodo_disciplina,disciplina_ID,nota
0,2014,1,0,5.872727
6,2014,2,0,5.625676
13,2015,1,0,6.126316
19,2015,2,0,5.486555
26,2016,1,0,4.139394
33,2016,2,0,5.750758


In [216]:
# Import BoxPlot, output_notebook, and show from bokeh.charts
from bokeh.charts import BoxPlot, output_notebook, show

# Make a box plot: p
p = BoxPlot(disciplinas_categorias, values='nota', label='disciplina_ID',
             title='Distribuição da média de notas das disciplinas',
             legend='bottom_right')

# Set the y axis label
p.yaxis.axis_label='Média por período)'

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...